## **SciAI: Density Optimization (Denop)**

Welcome to the SciAI Density Optimization notebook! With this notebook, you can perform density optimization using our pre-trained machine learning models. You pass in any molecule in the `.xyz` format, and get the optimized energy as output (among others).

Some more resources:

- [Structures25 paper. Stable and Accurate Orbital-Free Density Functional Theory Powered by Machine Learning](https://pubs.acs.org/doi/10.1021/jacs.5c06219)
- [SciAI Structures25 GitHub repository](https://github.com/sciai-lab/structures25)
- [The data that this Jupyter Notebook is downloading from HuggingFace](https://huggingface.co/sciai-lab/structures25/tree/main)

In [ ]:
import os

import torch

os.environ.setdefault("DFT_DATA", "")

DEVICE = "cpu"

if torch.cuda.is_available():
    DEVICE = "cuda"
    # if you work on a cluster with multiple GPUs, only use one of them
    # (because you are nice to your colleagues and don't want to take all of them)
    # import os
    # os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### 🎈 **Install & import dependencies**

In [ ]:
# Optional: install required packages
# %pip install -r requirements.txt -e .

In [ ]:
import torch
from omegaconf import DictConfig, OmegaConf
from pyscf import gto

from mldft.ml.models.mldft_module import MLDFTLitModule
from mldft.ofdft.functional_factory import FunctionalFactory
from mldft.ofdft.optimizer import TorchOptimizer
from mldft.ofdft.run_density_optimization import SampleGenerator, density_optimization
from mldft.utils.utils import set_default_torch_dtype

### 🎈 **Download our latest models from [Hugging Face](https://huggingface.co/sciai-lab/structures25/tree/main)**

If you want to run density optimization on your own molecules, just replace the `MOLECULE_FILE` variable below with your own file path.

In [ ]:
# https://huggingface.co/docs/datasets/cache#cache-directory
# The default cache directory is `~/.cache/huggingface/datasets`
# You can change it by setting this variable to any path you like
CACHE_DIR = None

In [ ]:
from huggingface_hub import hf_hub_download

# https://huggingface.co/sciai-lab/structures25/tree/main
REPO_ID = "sciai-lab/structures25"


def use_model_trained_on_qm9():
    print("Using QM9 model")
    model_config_path = hf_hub_download(
        repo_id=REPO_ID, filename="trained-on-qm9/hparams.yaml", cache_dir=CACHE_DIR
    )
    model_path = hf_hub_download(
        repo_id=REPO_ID,
        filename="trained-on-qm9/trained-on-qm9.ckpt",
        cache_dir=CACHE_DIR,
    )
    return model_config_path, model_path


def use_model_trained_on_qmugs():
    print("Using QMugs model")
    model_config_path = hf_hub_download(
        repo_id=REPO_ID, filename="trained-on-qmugs/hparams.yaml", cache_dir=CACHE_DIR
    )
    model_path = hf_hub_download(
        repo_id=REPO_ID,
        filename="trained-on-qmugs/trained-on-qmugs.ckpt",
        cache_dir=CACHE_DIR,
    )
    return model_config_path, model_path


# or alternatively: use_model_trained_on_qmugs()
MODEL_CONFIG_PATH, MODEL_PATH = use_model_trained_on_qm9()
MOLECULE_FILE = hf_hub_download(
    repo_id=REPO_ID, filename="molecule-samples/molecule.xyz", cache_dir=CACHE_DIR
)

### 🎈 **Run Denop (Density Optimization)**

In [ ]:
import numpy as np

from mldft.utils.plotting.plot_density_slices import PlotDensitySlices


@set_default_torch_dtype(torch.float64)
def denop(xyz_input_file: str, model_config_path: str, model_path: str):
    """
    Performs density optimization on the given molecule file (in .xyz format)
    using a pre-trained MLDFT model.
    """
    config = OmegaConf.load(model_config_path)
    assert isinstance(config, DictConfig), "Expected DictConfig"

    model = MLDFTLitModule.load_from_checkpoint(model_path, map_location=DEVICE)
    model.eval()
    model.to(torch.float64)

    optimizer = TorchOptimizer(
        torch.optim.SGD,  # pyright: ignore[reportPrivateImportUsage]
        convergence_tolerance=0.0001,
        max_cycle=10_000,
        lr=3e-3,
        momentum=0.9,
    )
    sample_generator = SampleGenerator(config, model)
    func_factory = FunctionalFactory.from_module(model)

    molecule = gto.M(atom=str(xyz_input_file))
    sample = sample_generator.get_sample_from_mol(molecule)

    print("🎈 Running density optimization")
    final_energies, final_coeffs, converged, _ = density_optimization(
        sample,
        sample.mol,
        optimizer,
        func_factory,
        initialization="sad_default",
        normalize_initial_guess=True,
    )

    if converged:
        print("✅ Density optimization converged")
        print("Final energies:", final_energies)
        # print("Final coeffs:", final_coeffs)

        # Visualize
        coeffs_numpy = (
            final_coeffs.detach().cpu().numpy()
            if hasattr(final_coeffs, "detach")
            else np.array(final_coeffs)
        )
        plot = PlotDensitySlices.show_mol_of_density(coeffs_numpy, sample.mol)
        plot.plot_difference()
    else:
        print("⚠️ Density optimization did not converge")


denop(MOLECULE_FILE, MODEL_CONFIG_PATH, MODEL_PATH)